In [ ]:
# 借鉴
# https://www.jianshu.com/p/7eed068ff353
# https://github.com/keras-team/keras/blob/master/examples/pretrained_word_embeddings.py
import os
import sys
import numpy as np
from gensim.models import word2vec, Word2Vec
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from keras.initializers import Constant
import tensorflow as tf

os.environ["CUDA_VISIBLE_DEVICES"] = "2"
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.3
set_session(tf.Session(config=config))

BASE_DIR = '/home/stu19/perl5/'
GLOVE_DIR = os.path.join(BASE_DIR, 'Golve_keras/glove.6B')
TEXT_DATA_DIR = os.path.join(BASE_DIR, 'Golve_keras/20_newsgroup')
MAX_SEQUENCE_LENGTH = 200
MAX_NUM_WORDS = 10000
EMBEDDING_DIM = 200
VALIDATION_SPLIT = 0.1

embeddings_index = {}
w2v_model = Word2Vec.load('./word2vec/w2v_200_mincount3.model')
for word in w2v_model.wv.vocab:
    coefs = np.asarray(w2v_model.wv[word], dtype='float32')
    embeddings_index[word] = coefs
print('Found %s word vectors,' % len(embeddings_index))

texts = []  # list of text samples 每个都是一个str
labels = []  # list of label ids
labels_num = 0
with open('/home/stu19/perl5/myself/train_set.csv', encoding = 'utf-8') as f1:
    for line in f1.readlines()[1:]:
        line = line.split(',')
        if str(int(line[3])-1) not in labels:
            labels_num += 1
        texts.append(line[2])
        labels.append(str(int(line[3])-1))
print("Found %s texts." % len(texts), "%s labels" % labels_num)

# finally, vectorize the text samples into a 2D integer tensor
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
print(labels[1])
labels = to_categorical(np.asarray(labels)) # label one hot，这里由于label是从1开始的。。。所以他会多出来一个0类型
print(labels[1])
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

# split the data into a training set and a validation set
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-num_validation_samples]
y_train = labels[:-num_validation_samples]
x_val = data[-num_validation_samples:]
y_val = labels[-num_validation_samples:]
print(data.shape, x_train.shape, y_train.shape, x_val.shape, y_val.shape)
print(x_train[1])

# prepare embedding matrix
num_words = min(MAX_NUM_WORDS, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i >= MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False) #so as to keep the embeddings fixed

# train a 1D convnet with global maxpooling
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(128, 5, activation='relu')(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = GlobalMaxPooling1D()(x)
x = Dense(128, activation='relu')(x)
preds = Dense(labels_num, activation='softmax')(x)

model = Model(sequence_input, preds)
# model.add(Dropout(0.2, input_shape=(60,)))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

model.fit(x_train, y_train,
          batch_size=256,
          epochs=100,
          validation_data=(x_val, y_val))

Found 498683 word vectors,
Found 102277 texts. 19 labels
